In [ ]:
!pip install rectools==0.2.0 implicit >> None

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import random
import requests
from rectools import Columns
from rectools.dataset import Dataset
from tqdm.auto import tqdm
import warnings
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel
warnings.filterwarnings("ignore")
import joblib

# Подгружаем данные KION 

In [ ]:
url = "https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip"

req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)
!unzip kion_train.zip

kion dataset download:   0%|          | 0.00/78.8M [00:00<?, ?iB/s]

Archive:  kion_train.zip
   creating: kion_train/
  inflating: kion_train/interactions.csv  
  inflating: __MACOSX/kion_train/._interactions.csv  
  inflating: kion_train/users.csv    
  inflating: __MACOSX/kion_train/._users.csv  
  inflating: kion_train/items.csv    
  inflating: __MACOSX/kion_train/._items.csv  


In [ ]:
interactions = pd.read_csv('kion_train/interactions.csv')
users = pd.read_csv('kion_train/users.csv')
items = pd.read_csv('kion_train/items.csv')

In [ ]:
interactions['last_watch_dt'] = pd.to_datetime(interactions['last_watch_dt'])

In [ ]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user_id        int64         
 1   item_id        int64         
 2   last_watch_dt  datetime64[ns]
 3   total_dur      int64         
 4   watched_pct    float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 208.9 MB


#Создаем аватаров

In [ ]:
# находим id последнего пользователя в датасете
last_orig_id = users['user_id'].max() # 1097558


In [ ]:
avatars_ids = [] # fake_user_ids

In [ ]:
avatar_id = last_orig_id + 1 # id следующего аватара будет равно следующее число после последнего пользователя
avatars_ids.append(avatar_id)

In [ ]:
avatars_ids

[1097559]

## Аватар "юморист"
предпочитает стендапы и комедии, мужчина 25-30 лет, с доходом 40-60 тыс.руб, без детей

In [ ]:
standup_ids = items[(items['genres']).str.contains('стендап')]['item_id'] #находим все id стендапов
standup_ids

106       6992
769       7403
1324     11647
1688      2818
1787      8017
1883     12555
2327     10915
3372      6345
6217      5353
6218     11648
8879     10358
10985     2878
11697    16513
12070     6734
12299    11079
13328     4652
13473    10094
13698    13176
13837    13168
13980     4265
14049    16002
Name: item_id, dtype: int64

In [ ]:
comedy_ids = items[(items['genres']).str.contains('комедии')]['item_id'] #находим все id комедий
comedy_ids

1         2508
2        10716
14        6181
15       15076
23       13109
         ...  
15929     4882
15931    10372
15932     3364
15942     3060
15962     3206
Name: item_id, Length: 3858, dtype: int64

In [ ]:
humour_ids = pd.concat([standup_ids, comedy_ids]) # объединяем все id для нашего юмориста

In [ ]:
humour_ids

106       6992
769       7403
1324     11647
1688      2818
1787      8017
         ...  
15929     4882
15931    10372
15932     3364
15942     3060
15962     3206
Name: item_id, Length: 3879, dtype: int64

In [ ]:
N = 10 # количество items, с которыми провзаимодействовал аватар

In [ ]:
# получаем взаимодействия аватара
avatar_interaction = humour_ids.sample(n=N, random_state=35) # выбираем N items, с которым провзаимодействовал аватар
last_watch_dt = ['2021-08-22'] * (N-1) 
last_watch_dt.append('2021-08-22')
total_dur = np.round(np.random.rand(N) * 10000 + 100).astype(dtype=np.int)
watched_pct = np.round(np.random.rand(N) * 100).astype(dtype=np.int)
data = {
    'user_id': [avatar_id] * N, # id для первого аватара
    'item_id': list(avatar_interaction),
    'last_watch_dt': last_watch_dt,
    'total_dur': list(total_dur),
    'watched_pct': list(watched_pct)
}
new_interactions = pd.DataFrame(data)
interactions = interactions.append(new_interactions) # получаем взаимодействия для аватара

In [ ]:
new_interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,1097559,15187,2021-08-22,5977,51
1,1097559,3305,2021-08-22,7379,50
2,1097559,9580,2021-08-22,1542,26
3,1097559,2792,2021-08-22,4369,45
4,1097559,6517,2021-08-22,3598,98
5,1097559,9929,2021-08-22,4717,74
6,1097559,8432,2021-08-22,6940,54
7,1097559,8438,2021-08-22,4155,66
8,1097559,12551,2021-08-22,9031,80
9,1097559,12282,2021-08-22,596,50


In [ ]:
avatar_features = {
    'user_id': [avatar_id],
    'age': ['age_25_34'],
    'income': ['income_40_60'],
    'sex': 'М',
    'kids_flg': [0]
}

new_user = pd.DataFrame(avatar_features)
users = users.append(new_user) # добавляем фичи аватара в датасет users


In [ ]:
new_user

,user_id,age,income,sex,kids_flg
0,1097559,age_25_34,income_40_60,М,0


## Аватар "семейный"
намешаны предпочтения мелодрамы, триллеры, мультфильмы; пусть аккаунт зарегистрирован на мужчину 35-44 лет, с доходом 60-90 тыс.руб, с детьми

In [ ]:
melodramas_ids = items[(items['genres']).str.contains('мелодрамы')]['item_id'] #находим все id мелодрам
melodramas_ids

0        10711
3         7868
4        16268
9         8604
10        3526
         ...  
15930     5473
15931    10372
15940     1922
15946     4430
15957    15610
Name: item_id, Length: 2778, dtype: int64

In [ ]:
thriller_ids = items[(items['genres']).str.contains('триллер')]['item_id'] #находим все id триллеров
thriller_ids

2        10716
8         9853
17        6881
21        3445
38        1622
         ...  
15923    10288
15939    15699
15945    12880
15958     6443
15960    10632
Name: item_id, Length: 2418, dtype: int64

In [ ]:
cartoon_ids = items[(items['genres']).str.contains('мультфильмы')]['item_id']
cartoon_ids

7        11114
30        8062
86       16045
111      13831
115       1964
         ...  
15859     4902
15892    13887
15915     5827
15949    15986
15950     6133
Name: item_id, Length: 571, dtype: int64

In [ ]:
family_ids = pd.concat([melodramas_ids, thriller_ids, cartoon_ids])

In [ ]:
len(family_ids)

5767

In [ ]:
# как как аватар семейный, то по логике должно быть больше взаимодействий
# возьмем 100
N = 100

In [ ]:
# получаем взаимодействия аватара
avatar_interaction = humour_ids.sample(n=N, random_state=35) # выбираем N items, с которым провзаимодействовал аватар
last_watch_dt = ['2021-07-22'] * (N-1) 
last_watch_dt.append('2021-08-22')
total_dur = np.round(np.random.rand(N) * 10000 + 100).astype(dtype=np.int)
watched_pct = np.round(np.random.rand(N) * 100).astype(dtype=np.int)
data = {
    'user_id': [avatar_id + 1] * N,
    'item_id': list(avatar_interaction),
    'last_watch_dt': last_watch_dt,
    'total_dur': list(total_dur),
    'watched_pct': list(watched_pct)
}
new_interactions = pd.DataFrame(data)
interactions = interactions.append(new_interactions) # получаем взаимодействия для аватара

In [ ]:
avatars_ids.append(avatar_id+1)

In [ ]:
avatars_ids

[1097559, 1097560]

In [ ]:
new_interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,1097560,15187,2021-07-22,3197,68
1,1097560,3305,2021-07-22,4051,46
2,1097560,9580,2021-07-22,1490,83
3,1097560,2792,2021-07-22,2108,56
4,1097560,6517,2021-07-22,9452,80
...,...,...,...,...,...
95,1097560,13561,2021-07-22,3308,63
96,1097560,6192,2021-07-22,2381,23
97,1097560,141,2021-07-22,9961,98
98,1097560,6214,2021-07-22,2808,15


In [ ]:
avatar_features = {
    'user_id': [avatar_id],
    'age': ['age_35_44'],
    'income': ['income_60_90'],
    'sex': 'М',
    'kids_flg': [1]
}

new_user = pd.DataFrame(avatar_features)
users = users.append(new_user) # добавляем фичи аватара в датасет users

In [ ]:
new_user

,user_id,age,income,sex,kids_flg
0,1097559,age_35_44,income_60_90,М,1


## Любитель "пощекотать нервы"
предпочитает ужасы, мужчина 18-24 года, доход 40-60 тыс, без детей

In [ ]:
horror_ids = items[(items['genres']).str.contains('ужасы')]['item_id'] #находим все id ужасов
horror_ids

15       15076
16        2904
17        6881
49        9787
50       11593
         ...  
15862    16419
15877     9497
15905     6245
15910    16200
15937     7805
Name: item_id, Length: 1033, dtype: int64

In [ ]:
N = 25

In [ ]:
# получаем взаимодействия аватара
avatar_interaction = humour_ids.sample(n=N, random_state=35) # выбираем N items, с которым провзаимодействовал аватар
last_watch_dt = ['2021-06-22'] * (N-1) 
last_watch_dt.append('2021-08-22')
total_dur = np.round(np.random.rand(N) * 10000 + 100).astype(dtype=np.int)
watched_pct = np.round(np.random.rand(N) * 100).astype(dtype=np.int)
data = {
    'user_id': [avatar_id+2] * N,
    'item_id': list(avatar_interaction),
    'last_watch_dt': last_watch_dt,
    'total_dur': list(total_dur),
    'watched_pct': list(watched_pct)
}
new_interactions = pd.DataFrame(data)
interactions = interactions.append(new_interactions) # получаем взаимодействия для аватара

In [ ]:
avatars_ids.append(avatar_id+2)

In [ ]:
# Список id ававтаров
avatars_ids 

[1097559, 1097560, 1097561]

In [ ]:
new_interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,1097561,15187,2021-06-22,711,65
1,1097561,3305,2021-06-22,5988,58
2,1097561,9580,2021-06-22,368,86
3,1097561,2792,2021-06-22,1544,38
4,1097561,6517,2021-06-22,1718,22
5,1097561,9929,2021-06-22,4323,27
6,1097561,8432,2021-06-22,1439,57
7,1097561,8438,2021-06-22,4310,77
8,1097561,12551,2021-06-22,8609,54
9,1097561,12282,2021-06-22,212,60


In [ ]:
avatar_features = {
    'user_id': [avatar_id],
    'age': ['age_18_24'],
    'income': ['income_40_60'],
    'sex': 'М',
    'kids_flg': [0]
}

new_user = pd.DataFrame(avatar_features)
users = users.append(new_user) # добавляем фичи аватара в датасет users

In [ ]:
new_user

,user_id,age,income,sex,kids_flg
0,1097559,age_18_24,income_40_60,М,0


In [ ]:
users.to_csv('/content/drive/MyDrive/ITMO_MTC_course/hw_4/users_with_avatars.csv', index=False)
interactions.to_csv('/content/drive/MyDrive/ITMO_MTC_course/hw_4/interactions_with_avatars.csv', index=False)

# Проверка модели на аватарах

In [ ]:
interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11 00:00:00,4250,72.0
1,699317,1659,2021-05-29 00:00:00,8317,100.0
2,656683,7107,2021-05-09 00:00:00,10,0.0
3,864613,7638,2021-07-05 00:00:00,14483,100.0
4,964868,9506,2021-04-30 00:00:00,6725,100.0
...,...,...,...,...,...
20,1097561,672,2021-06-22,3894,33.0
21,1097561,6820,2021-06-22,10067,16.0
22,1097561,9606,2021-06-22,7811,89.0
23,1097561,3236,2021-06-22,4861,97.0


In [ ]:
K_RECOS = 10

In [ ]:
model = joblib.load('/content/drive/MyDrive/Проекты/recsys/models/LightFM_warp_16.joblib')

In [ ]:
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight}, 
                    inplace=True) 
dataset = Dataset.construct(interactions_df=interactions)

In [ ]:
model.fit(dataset)

## Проверка рекомендаций для юмориста

In [ ]:
recos = model.recommend(users=[1097559], 
                        dataset=dataset, 
                        k=K_RECOS, 
                        filter_viewed=True)

In [ ]:
items[items['item_id'].isin(recos['item_id'])]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
767,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча..."
927,11778,film,Простые сложности,It's Complicated,2009.0,"драмы, мелодрамы, комедии","США, Япония",NaN,16.0,NaN,Нэнси Майэрс,"Мэрил Стрип, Алек Болдуин, Стив Мартин, Джон К...","Режиссёрка Нэнси Майерс, известная благодаря ф...","Нью-Йорк, отель, жених, выпускной, бывший муж,..."
1882,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе..."
4730,6192,series,Отчаянные домохозяйки,Desperate Housewives,2004.0,"драмы, мелодрамы, детективы, комедии",США,NaN,16.0,NaN,"Дэвид Гроссман, Ларри Шоу, Дэвид Уоррен","Тери Хэтчер, Фелисити Хаффман, Марсия Кросс, Е...",В центре событий - четыре современные домохозя...,"секрет, домохозяйка, женская дружба, сосед, со..."
6346,9728,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры","Великобритания, США",NaN,18.0,NaN,Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер..."
6501,13865,film,Девятаев,V2. Escape from Hell,2021.0,"драмы, военные, приключения",Россия,NaN,12.0,NaN,Тимур Бекмамбетов,"Павел Прилучный, Павел Чинарёв, Тимофей Трибун...",Военно-исторический блокбастер от режиссёров Т...,"Девятаев, Девятаева, Девят, Девя, Девята, Девя..."
6689,4151,series,Секреты семейной жизни,NaN,2021.0,комедии,Россия,NaN,18.0,NaN,Шота Гамисония,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...",У Никиты и Полины всё начиналось прекрасно: об...,"брызги крови, кровь, жестокое обращение с живо..."
10604,12192,series,Фемида видит,Femida vidit,2019.0,"драмы, детективы, комедии",Россия,NaN,16.0,NaN,"Никита Грамматиков, Ирина Бас","Анна Котова-Дерябина, Александр Давыдов, Алекс...",Лёгкий комедийный детектив от автора «Подслуша...,"фемида видит, фемида, феми, фем, фемид, фемида..."
12050,3734,film,Прабабушка легкого поведения,Prababushka lyogkogo povedeniya,2021.0,комедии,Россия,NaN,16.0,NaN,Марюс Вайсберг,"Александр Ревва, Глюкоза, Дмитрий Нагиев, Миха...","1980 год, вся страна следит за событиями моско...",", 2021, россия, прабабушка, легкого, поведения"
13615,2657,series,Подслушано,Podslushano,2021.0,"драмы, триллеры",Россия,NaN,16.0,NaN,Илья Куликов,"Александр Hовиков, Валентина Ляпина, Никита Па...",Смотри:- как кино- как сериалПодростковый псих...,"подслушано, подслушано в контакте, социальная ..."


В целом хороший результат для юмориста, комедии разбавлены драмами и боевиками

## Проверка рекомендаций для семейного

In [ ]:
recos = model.recommend(users=[1097560], 
                        dataset=dataset, 
                        k=K_RECOS, 
                        filter_viewed=True)

In [ ]:
items[items['item_id'].isin(recos['item_id'])]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
767,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча..."
1663,11899,film,Вся правда о любви,The Truth About Love,2005.0,"мелодрамы, комедии",Великобритания,NaN,16.0,NaN,Джон Хэй,"Дюгрей Скотт, Дженнифер Лав Хьюитт, Дэвид Крис...",Желая проверить верность и искренность чувств ...,"2005, соединенное королевство, вся, правда, любви"
4118,626,film,"Окей, Лекси!",Jexi,2019.0,"фантастика, мелодрамы, комедии","США, Канада",NaN,18.0,NaN,"Джон Лукас, Скотт Мур","Адам Дивайн, Александра Шипп, Роуз Бирн, Рон Ф...",Фил вырос с телефоном в руках - так постоянно ...,"искусственный интеллект, Сан-Франциско, Калифо..."
6111,2820,film,Я ненавижу день Святого Валентина,I Hate Valentine's Day,2009.0,"мелодрамы, комедии",США,NaN,16.0,NaN,Ниа Вардалос,"Ниа Вардалос, Джон Корбетт, Стефен Джуарино, А...","Девушка-флорист хочет, чтобы ее пригласил на с...","2009, соединенные штаты, ненавижу, день, свято..."
8366,9343,film,Ассистент звезды,"High note, The",2020.0,"драмы, мелодрамы","США, Великобритания",NaN,12.0,NaN,Ниша Ганатра,"Дакота Джонсон, Трэйси Эллис Росс, Келвин Харр...",С самого детства музыка играла важную роль в ж...,"2020, соединенные штаты, соединенное королевст..."
8711,11188,film,"С любовью, Рози","Love, Rosie",2014.0,"мелодрамы, комедии","Германия, Великобритания",NaN,16.0,NaN,Кристиан Диттер,"Лили Коллинз, Сэм Клафлин, Кристиан Кук, Джеми...","Рози и Алекс были лучшими друзьями с детства, ...","дружба, лучшая подруга, подростковая беременно..."
9181,15171,film,Все пары делают это,Daddy Cool,2017.0,комедии,Франция,NaN,16.0,NaN,Максим Говаре,"Венсан Эльбаз, Лоранс Арне, Жан Франсуа Кэйри,...","«Я хочу завести детей, но у меня уже есть один...","2017, франция, все, пары, делают, это"
9782,4030,film,Человеческий голос,Human Voice,2020.0,драмы,Испания,NaN,16.0,NaN,Педро Альмодовар,"Тильда Суинтон, Агустин Альмодовар, Пабло Альм...",Женщина замерла в надежде на возвращение бывше...,"короткометражка, 2020, испания, человеческий, ..."
14162,11228,film,Бриджит Джонс 3,Bridget Jones's Baby,2016.0,"драмы, мелодрамы, комедии","США, Франция, Китай, Великобритания",NaN,16.0,NaN,Шэрон Магуайр,"Рене Зеллвегер, Колин Фёрт, Патрик Демпси, Дже...","Казалось бы, и карьера – в гору, и квартира – ...","любовный треугольник, малыш, беременность, про..."
15153,6774,film,Тачки 2,Cars 2,2011.0,"мультфильм, комедии",США,NaN,0.0,NaN,"Джон Лассетер, Брэд Льюис","Оуэн Уилсон, Ларри-кабельщик, Майкл Кейн, Эмил...",Молния МакКуин и его друг Мэтр отправляются в ...,"автомобильная гонка, продолжение, антропоморфи..."


Для типа семейный замечательно подобраны рекомендации

## Проверка рекомендаций для любителя пощекотать нервы

In [ ]:
recos = model.recommend(users=[1097561], 
                        dataset=dataset, 
                        k=K_RECOS, 
                        filter_viewed=True)

In [ ]:
items[items['item_id'].isin(recos['item_id'])]

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
767,15297,series,Клиника счастья,Klinika schast'ya,2021.0,"драмы, мелодрамы",Россия,NaN,18.0,NaN,Александр Кириенко,"Дарья Мороз, Анатолий Белый, Данил Акутин, Мар...","Успешный сексолог Алена уверена, что нашла фор...","Клиника счастья, Клиника, Счастье, Клиника сча..."
1274,5411,film,Монстры на каникулах 3: Море зовёт,Hotel Transylvania 3: Summer Vacation,2018.0,"мультфильм, фэнтези, приключения, комедии",США,NaN,6.0,NaN,Генндий Тартаковский,"Энди Сэмберг, Селена Гомес, Кевин Джеймс, Стив...",На этот раз Дракула вместе со своими друзьями ...,"третья часть, круизное судно, персонаж Дракула..."
1849,7571,film,100% волк,100% Wolf,2020.0,"мультфильм, приключения, семейное, фэнтези, ко...","Австралия, Бельгия",NaN,6.0,NaN,Алекс Стадерманн,"Илай Суинделлс, Самара Уивинг, Джай Кортни, Ру...",Наследник семьи оборотней Фредди Люпин отчаянн...,"пудель, подростковая тревога, оборотень, приня..."
1882,10440,series,Хрустальный,Khrustal'nyy,2021.0,"триллеры, детективы",Россия,NaN,18.0,NaN,Душан Глигоров,"Антон Васильев, Николай Шрайбер, Екатерина Оль...",Сергей Смирнов — один из лучших «охотников на ...,"хруст, хрусталь, хруста, хрус, полицейский, пе..."
3951,3182,film,Ральф против Интернета,Ralph Breaks the Internet,2018.0,"мультфильм, приключения, фантастика, семейное,...",США,NaN,6.0,NaN,"Рич Мур, Фил Джонстон","Джон Си Райли, Сара Силверман, Галь Гадот, Тар...",На этот раз Ральф и Ванилопа фон Кекс выйдут з...,"видеоигра, мультфильм, продолжение, интернет, ..."
6346,9728,film,Гнев человеческий,Wrath of Man,2021.0,"боевики, триллеры","Великобритания, США",NaN,18.0,NaN,Гай Ричи,"Джейсон Стэйтем, Холт МакКэллани, Джеффри Доно...",Грузовики лос-анджелесской инкассаторской комп...,"ограбление, криминальный авторитет, месть, пер..."
6689,4151,series,Секреты семейной жизни,NaN,2021.0,комедии,Россия,NaN,18.0,NaN,Шота Гамисония,"Петр Скворцов, Алена Михайлова, Федор Лавров, ...",У Никиты и Полины всё начиналось прекрасно: об...,"брызги крови, кровь, жестокое обращение с живо..."
8711,11188,film,"С любовью, Рози","Love, Rosie",2014.0,"мелодрамы, комедии","Германия, Великобритания",NaN,16.0,NaN,Кристиан Диттер,"Лили Коллинз, Сэм Клафлин, Кристиан Кук, Джеми...","Рози и Алекс были лучшими друзьями с детства, ...","дружба, лучшая подруга, подростковая беременно..."
8854,12360,film,Круэлла,Cruella,2021.0,"драмы, криминал, комедии","США, Великобритания",NaN,12.0,NaN,Крэйг Гиллеспи,"Эмма Стоун, Эмма Томпсон, Джоэль Фрай, Пол Уол...","Великобритания, 1960-е годы. Эстелла была необ...","1970-е, антигерой, злодей, панк-рок, модельер,..."
9698,16166,film,Зверополис,Zootopia,2016.0,"приключения, мультфильм, детективы, комедии",США,NaN,6.0,NaN,"Байрон Ховард, Рич Мур, Джаред Буш","Джиннифер Гудвин, Джейсон Бейтман, Идрис Эльба...",Добро пожаловать в Зверополис — современный го...,"аллегория, лев, бегемот, лиса, слон, овца, бел..."


Рекомендации для любителя пощекотать нервы подобраны не очень хорошо, возможно модель с фичами показала бы лучший результат